<a href="https://colab.research.google.com/github/PBuenoc/f1ProjectInGoogleColab/blob/main/F1ProjectInColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare spark environment

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName("Spark").getOrCreate()

# Ingestion

## Ingest csv files

### Ingest circuits.csv file

In [ ]:
circuits_df = spark.read.csv('/content/drive/MyDrive/formula1Project/data/raw/circuits2.csv',inferSchema=True, header=True, sep=';')

#### Select only the columns required

In [ ]:
circuits_df = circuits_df.select('circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng', 'alt')

#### Renamed the columns as required

In [ ]:
circuits_df = circuits_df.withColumnRenamed('circuitId', 'circuit_id') \
.withColumnRenamed('circuitRef', 'circuit_ref') \
.withColumnRenamed('lat','latitude') \
.withColumnRenamed('lng','longitude') \
.withColumnRenamed('alt','altitude')

#### Add ingestion_date column

In [ ]:
from pyspark.sql.functions import current_timestamp

In [ ]:
circuits_df = circuits_df.withColumn('ingestion_date', current_timestamp())

#### Specify the types as required

In [ ]:
circuits_df = circuits_df.withColumn('latitude', circuits_df['latitude'].cast('double')) \
.withColumn('longitude', circuits_df['longitude'].cast('double')) \
.withColumn('altitude', circuits_df['altitude'].cast('integer'))

#### Write the data in parquet format on processed folder

In [ ]:
circuits_df.write.mode('overwrite').parquet('/content/drive/MyDrive/formula1Project/data/processed/circuits')

### Ingest races.csv

In [ ]:
races_df = spark.read.csv('/content/drive/MyDrive/formula1Project/data/raw/races2.csv', header=True, inferSchema=True, sep=';')

#### Add the required columns

In [ ]:
from pyspark.sql.functions import col, concat, lit

In [ ]:
races_df = races_df.withColumn('ingestion_date', current_timestamp()) \
.withColumn('race_timestamp',concat(col('date'), lit(' '), col('time')))

#### Select only the required columns

In [ ]:
races_df = races_df.select(col('raceId').alias('race_id'),
                           col('year').alias('race_year'), 
                           col('round'), 
                           col('circuitId').alias('circuit_id'),
                           col('name'),
                           col('ingestion_date'),
                           col('race_timestamp'))

#### Write the data in parquet format on processed folder with partitionBy

In [ ]:
races_df.write.mode('overwrite').partitionBy('race_year').parquet('/content/drive/MyDrive/formula1Project/data/processed/races')

In [ ]:
spark.read.parquet('/content/drive/MyDrive/formula1Project/data/processed/races').show()

+-------+-----+----------+--------------------+--------------------+-------------------+---------+
|race_id|round|circuit_id|                name|      ingestion_date|     race_timestamp|race_year|
+-------+-----+----------+--------------------+--------------------+-------------------+---------+
|   1053|    2|        21|Emilia Romagna Gr...|2023-03-23 21:23:...|18/04/2021 13:00:00|     2021|
|   1052|    1|         3|  Bahrain Grand Prix|2023-03-23 21:23:...|28/03/2021 15:00:00|     2021|
|   1051|   21|         1|Australian Grand ...|2023-03-23 21:23:...|21/11/2021 06:00:00|     2021|
|   1054|    3|        20|                 TBC|2023-03-23 21:23:...|      02/05/2021 \N|     2021|
|   1055|    4|         4|  Spanish Grand Prix|2023-03-23 21:23:...|09/05/2021 13:00:00|     2021|
|   1056|    5|         6|   Monaco Grand Prix|2023-03-23 21:23:...|23/05/2021 13:00:00|     2021|
|   1057|    6|        73|Azerbaijan Grand ...|2023-03-23 21:23:...|06/06/2021 12:00:00|     2021|
|   1058| 

## Ingest JSON files

### Ingest constructors.json

In [ ]:
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [ ]:
constructors_df = spark.read.json('/content/drive/MyDrive/formula1Project/data/raw/constructors.json', schema=constructors_schema)

#### Drop unwanted columns from the dataframe

In [ ]:
constructors_df = constructors_df.drop(constructors_df.url)

#### Rename columns and add ingestion date

In [ ]:
constructors_df = constructors_df.withColumnRenamed('constructorId', 'constructor_id') \
                                .withColumnRenamed('constructorRef', 'constructor_ref') \
                                .withColumn('ingestion_date', current_timestamp())

In [ ]:
constructors_df.write.mode('overwrite').parquet('/content/drive/MyDrive/formula1Project/data/processed/constructors')

### Ingest drivers.json - Nested JSON

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [ ]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
  
])

In [ ]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)])

In [ ]:
drivers_df = spark.read \
.schema(drivers_schema) \
.json('/content/drive/MyDrive/formula1Project/data/raw/drivers.json')

In [ ]:
from google.colab import data_table

In [ ]:
from pyspark.sql.functions import col, concat, lit

In [ ]:
drivers_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

In [ ]:
from pyspark.sql.functions import current_timestamp

In [ ]:
drivers_df = drivers_df.withColumn('ingestion_date', current_timestamp())

#### Drop unwanted columns
url \
name.forename \
name.surname \

In [ ]:
drivers_df = drivers_df.drop('url')

#### Write the output to processed folder in parquet format


In [ ]:
drivers_df.write.mode('overwrite').parquet('/content/drive/MyDrive/formula1Project/data/processed/drivers')

In [ ]:
spark.read.parquet('/content/drive/MyDrive/formula1Project/data/processed/drivers').show()

+---------+----------+------+----+------------------+----------+-----------+--------------------+
|driver_id|driver_ref|number|code|              name|       dob|nationality|      ingestion_date|
+---------+----------+------+----+------------------+----------+-----------+--------------------+
|        1|  hamilton|    44| HAM|    Lewis Hamilton|1985-01-07|    British|2023-03-23 21:26:...|
|        2|  heidfeld|  null| HEI|     Nick Heidfeld|1977-05-10|     German|2023-03-23 21:26:...|
|        3|   rosberg|     6| ROS|      Nico Rosberg|1985-06-27|     German|2023-03-23 21:26:...|
|        4|    alonso|    14| ALO|   Fernando Alonso|1981-07-29|    Spanish|2023-03-23 21:26:...|
|        5|kovalainen|  null| KOV| Heikki Kovalainen|1981-10-19|    Finnish|2023-03-23 21:26:...|
|        6|  nakajima|  null| NAK|   Kazuki Nakajima|1985-01-11|   Japanese|2023-03-23 21:26:...|
|        7|  bourdais|  null| BOU|Sébastien Bourdais|1979-02-28|     French|2023-03-23 21:26:...|
|        8| raikkone

### Ingest results.json

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [ ]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [ ]:
results_df = spark.read \
.schema(results_schema) \
.json('/content/drive/MyDrive/formula1Project/data/raw/results.json')

#### Drop, rename and add required columns

In [ ]:
results_df = results_df.drop('statusId')

In [ ]:
results_df = results_df.withColumnRenamed('resultId', 'result_id') \
                       .withColumnRenamed('raceId', 'race_id') \
                       .withColumnRenamed('driverId', 'driver_id') \
                       .withColumnRenamed('constructorId', 'constructor_id') \
                       .withColumnRenamed('positionText', 'position_text') \
                       .withColumnRenamed('positionOrder', 'position_order') \
                       .withColumnRenamed('fastestLap', 'fastest_lap') \
                       .withColumnRenamed('fastestLapTime', 'fastest_lap_time') \
                       .withColumnRenamed('fastestLapSpeed', 'fastest_lap_speed')

In [ ]:
from pyspark.sql.functions import current_timestamp

In [ ]:
results_df = results_df.withColumn('ingestion_date', current_timestamp())

#### Write the output to processed folder in parquet format

In [ ]:
results_df.write.mode('overwrite').partitionBy('race_id').parquet('/content/drive/MyDrive/formula1Project/data/processed/results')


### Ingest pitstops.json

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
pit_stops_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("stop", StringType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("duration", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                     ])

In [ ]:
pit_stops_df = spark.read \
.schema(pit_stops_schema) \
.option('multiline', True) \
.json('/content/drive/MyDrive/formula1Project/data/raw/pit_stops.json')

#### Rename and add columns as required

In [ ]:
pit_stops_df = pit_stops_df.withColumnRenamed('raceId', 'race_id') \
                           .withColumnRenamed('driverId', 'driver_id')

In [ ]:
pit_stops_df = pit_stops_df.withColumn('ingestion_date', current_timestamp())

#### Write the output to processed folder in parquet format

In [ ]:
pit_stops_df.write.mode("overwrite").parquet('/content/drive/MyDrive/formula1Project/data/processed/pit_stops')

In [ ]:
spark.read.parquet('/content/drive/MyDrive/formula1Project/data/processed/pit_stops').toPandas()

,race_id,driver_id,stop,lap,time,duration,milliseconds,ingestion_date
0,841,153,1,1,17:05:23,26.898,26898,2023-03-23 21:34:19.103
1,841,30,1,1,17:05:52,25.021,25021,2023-03-23 21:34:19.103
2,841,17,1,11,17:20:48,23.426,23426,2023-03-23 21:34:19.103
3,841,4,1,12,17:22:34,23.251,23251,2023-03-23 21:34:19.103
4,841,13,1,13,17:24:10,23.842,23842,2023-03-23 21:34:19.103
...,...,...,...,...,...,...,...,...
8025,1047,20,1,35,18:17:15,22.04,22040,2023-03-23 21:34:19.103
8026,1047,849,2,35,18:17:41,22.384,22384,2023-03-23 21:34:19.103
8027,1047,817,1,39,18:23:37,22.123,22123,2023-03-23 21:34:19.103
8028,1047,825,2,47,18:39:11,23.098,23098,2023-03-23 21:34:19.103


## Ingest multiple files

### Ingest lap_times files - Multiple CSV files

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [ ]:
lap_times_schema = StructType(fields=[StructField('raceId', IntegerType(), False),
                                      StructField('driverId', IntegerType(), False),
                                      StructField('lap', IntegerType(), False),
                                      StructField('position', IntegerType(), True),
                                      StructField('time', StringType(), True),
                                      StructField('milliseconds', IntegerType(), True),
])

In [ ]:
lap_times_df = spark.read \
.schema(lap_times_schema) \
.csv('/content/drive/MyDrive/formula1Project/data/raw/lap_times')

In [ ]:
from pyspark.sql.functions import current_timestamp, from_utc_timestamp

In [ ]:
lap_times_df = lap_times_df.withColumnRenamed('raceId', 'race_id') \
                           .withColumnRenamed('driverId', 'driver_id') \
                           .withColumn("ingestion_date", from_utc_timestamp(current_timestamp(), "GMT-3"))

In [ ]:
lap_times_df.write.mode('overwrite').parquet('/content/drive/MyDrive/formula1Project/data/processed/lap_times')

In [ ]:
spark.read.parquet('/content/drive/MyDrive/formula1Project/data/processed/lap_times').limit(11).toPandas()

,race_id,driver_id,lap,position,time,milliseconds,ingestion_date
0,841,20,1,1,1:38.109,98109,2023-03-23 18:29:54.668
1,841,20,2,1,1:33.006,93006,2023-03-23 18:29:54.668
2,841,20,3,1,1:32.713,92713,2023-03-23 18:29:54.668
3,841,20,4,1,1:32.803,92803,2023-03-23 18:29:54.668
4,841,20,5,1,1:32.342,92342,2023-03-23 18:29:54.668
5,841,20,6,1,1:32.605,92605,2023-03-23 18:29:54.668
6,841,20,7,1,1:32.502,92502,2023-03-23 18:29:54.668
7,841,20,8,1,1:32.537,92537,2023-03-23 18:29:54.668
8,841,20,9,1,1:33.240,93240,2023-03-23 18:29:54.668
9,841,20,10,1,1:32.572,92572,2023-03-23 18:29:54.668


### Ingest qualifying files - Multiple MultIline JSON files

In [ ]:
qualifying_schema = StructType(fields=[StructField('qualifyId', IntegerType(), False),
                                         StructField('raceId', IntegerType(), True),
                                         StructField('driverId', IntegerType(), True),
                                         StructField('constructorId', IntegerType(), True),
                                         StructField('number', IntegerType(), True),
                                         StructField('position', IntegerType(), True),
                                         StructField('q1', StringType(), True),
                                         StructField('q2', StringType(), True),
                                         StructField('q3', StringType(), True),
  
])

In [ ]:
qualifying_df = spark.read \
.schema(qualifying_schema) \
.option('multiLine', True) \
.json('/content/drive/MyDrive/formula1Project/data/raw/qualifying/*')

In [ ]:
qualifying_df = qualifying_df.withColumnRenamed('qualifyId', 'qualify_id') \
                             .withColumnRenamed('raceId', 'race_id') \
                             .withColumnRenamed('driverId', 'driver_id') \
                             .withColumnRenamed('constructorId', 'constructor_id') \
                             .withColumn('ingestion_date', current_timestamp())

In [ ]:
qualifying_df.write.mode('overwrite').parquet('/content/drive/MyDrive/formula1Project/data/processed/qualifying')

# Filter & Joins Transformations

## Read and renamed

In [11]:
races_df = spark.read.parquet('/content/drive/MyDrive/formula1Project/data/processed/races') \
.withColumnRenamed('race_timestamp', 'race_date') \
.withColumnRenamed('name', 'race_name')

In [20]:
circuits_df = spark.read.parquet('/content/drive/MyDrive/formula1Project/data/processed/circuits') \
.withColumnRenamed('location', 'circuit_location')

In [17]:
constructors_df = spark.read.parquet('/content/drive/MyDrive/formula1Project/data/processed/constructors') \
.withColumnRenamed('name', 'team')

In [19]:
results_df = spark.read.parquet('/content/drive/MyDrive/formula1Project/data/processed/results') \
.withColumnRenamed('time', 'race_time')

In [15]:
drivers_df = spark.read.parquet('/content/drive/MyDrive/formula1Project/data/processed/drivers') \
.withColumnRenamed('number', 'driver_number') \
.withColumnRenamed('name', 'driver_name') \
.withColumnRenamed('nationality', 'driver_nationality')

## Join circuits to races

In [32]:
race_circuit_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, 'inner').select('race_id','race_year', 'race_name', 'race_date', 'circuit_location') 

In [33]:
race_circuit_df.show()

+-------+---------+--------------------+-------------------+----------------+
|race_id|race_year|           race_name|          race_date|circuit_location|
+-------+---------+--------------------+-------------------+----------------+
|   1053|     2021|Emilia Romagna Gr...|18/04/2021 13:00:00|           Imola|
|   1052|     2021|  Bahrain Grand Prix|28/03/2021 15:00:00|          Sakhir|
|   1051|     2021|Australian Grand ...|21/11/2021 06:00:00|       Melbourne|
|   1054|     2021|                 TBC|      02/05/2021 \N|         Nürburg|
|   1055|     2021|  Spanish Grand Prix|09/05/2021 13:00:00|        Montmeló|
|   1056|     2021|   Monaco Grand Prix|23/05/2021 13:00:00|     Monte-Carlo|
|   1057|     2021|Azerbaijan Grand ...|06/06/2021 12:00:00|            Baku|
|   1058|     2021| Canadian Grand Prix|13/06/2021 18:00:00|        Montreal|
|   1059|     2021|   French Grand Prix|27/06/2021 13:00:00|    Le Castellet|
|   1060|     2021| Austrian Grand Prix|04/07/2021 13:00:00|    

## Join result to all other df's

In [37]:
race_results_df = results_df.join(race_circuit_df, results_df.race_id == race_circuit_df.race_id) \
                            .join(drivers_df, results_df.driver_id == drivers_df.driver_id) \
                            .join(constructors_df, results_df.constructor_id == constructors_df.constructor_id)

In [39]:
from pyspark.sql.functions import current_timestamp

In [43]:
final_df = race_results_df.select('race_year', 'race_name', 'race_date', 'circuit_location', 'driver_name', 'driver_number', 'driver_nationality', 'team', 'grid', 'fastest_lap', 'race_time', 'points') \
.withColumn('created_date', current_timestamp())

In [47]:
final_df.where("race_year == 2020 and race_name == 'Abu Dhabi Grand Prix'").orderBy(final_df.points.desc()).limit(11).toPandas()

,race_year,race_name,race_date,circuit_location,driver_name,driver_number,driver_nationality,team,grid,fastest_lap,race_time,points,created_date
0,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Max Verstappen,33,Dutch,Red Bull,1,14,1:36:28.645,25.0,2023-03-25 03:01:43.061
1,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Valtteri Bottas,77,Finnish,Mercedes,2,40,+15.976,18.0,2023-03-25 03:01:43.061
2,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Lewis Hamilton,44,British,Mercedes,3,37,+18.415,15.0,2023-03-25 03:01:43.061
3,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Alexander Albon,23,Thai,Red Bull,5,42,+19.987,12.0,2023-03-25 03:01:43.061
4,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Lando Norris,4,British,McLaren,4,53,+1:00.729,10.0,2023-03-25 03:01:43.061
5,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Carlos Sainz,55,Spanish,McLaren,6,48,+1:05.662,8.0,2023-03-25 03:01:43.061
6,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Daniel Ricciardo,3,Australian,Renault,11,55,+1:13.748,7.0,2023-03-25 03:01:43.061
7,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Pierre Gasly,10,French,AlphaTauri,9,53,+1:29.718,4.0,2023-03-25 03:01:43.061
8,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Esteban Ocon,31,French,Renault,10,47,+1:41.069,2.0,2023-03-25 03:01:43.061
9,2020,Abu Dhabi Grand Prix,13/12/2020 13:10:00,Abu Dhabi,Lance Stroll,18,Canadian,Racing Point,8,41,+1:42.738,1.0,2023-03-25 03:01:43.061


In [50]:
final_df.write.mode('overwrite').parquet('/content/drive/MyDrive/formula1Project/data/presentation/race_results')